In [1]:
import logging
from pathlib import Path
import numpy as np
import pandas as pd
import re
from gensim import corpora, models,matutils
import random
from tqdm import tqdm_notebook as tqdm
import sys

import notebookutil as nbu
sys.meta_path.append(nbu.NotebookFinder())
from util import load_data_and_labels



from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

%load_ext jupyternotify

Japanese stopword:  あそこ, あたり, あちら ...
English stopword: ... you've, z, zero


<IPython.core.display.Javascript object>

# Character level + random forest

In [2]:
positive_data_file = "data/amazon_ja/pos.txt"
negative_data_file = "data/amazon_ja/neg.txt"

In [3]:
files = ["data/amazon_ja/r_{}.txt".format(i) for i in range(1,6)]

In [10]:
%%notify
level = "char" #"char"
x_text, y, ratio = load_data_and_labels(positive_data_file, negative_data_file, level=level, lang="Ja")
#level="char"
#x_text, y = load_data_and_labels_multiclass(files, level=level, lang="Ja")

# pos:  62402
# neg:  9060
pos/neg: 6.887637969094922


<IPython.core.display.Javascript object>

In [12]:
x_text[0]

' 書 き 込 み 、 読 み 出 し 、 転 送 速 度 、 い ず れ も 満 足 で す 。 1 6 0 0 万 画 素 の コ ン パ ク ト カ メ ラ タ イ プ の デ ジ カ メ に 入 れ て 撮 影 に 使 い 、 撮 影 後 は カ ー ド リ ー ダ に 接 続 し て 、 撮 影 し た 膨 大 な 量 の 画 像 デ ー タ を サ ム ネ イ ル 表 示 に し た り 、 ピ ッ ク ア ッ プ し た 画 像 を コ ピ ペ し た り し て い ま す が 、 と く に ス ト レ ス を 感 じ る こ と な く 、 快 適 に 使 え て い ま す 。 【 a m a z o n . c o . j p 限 定 】 の 個 体 は 、 s d カ ー ド 本 体 が シ ン プ ル な 小 さ い ボ ー ル 紙 に 挟 ま れ て い る だ け 。 梱 包 は 超 シ ン プ ル で す が 、 実 売 価 格 が 安 く 、 性 能 に も 満 足 出 来 て い る の で 買 っ て 良 か っ た と 思 っ て い ま す 。 耐 久 性 は わ か り ま せ ん 。 そ こ は 要 経 過 観 察 で す ね 。 \n '

In [13]:
x_text_sp = [doc[:-2].split() for doc in x_text]

In [14]:
%%notify
d = corpora.Dictionary(x_text_sp)

<IPython.core.display.Javascript object>

In [15]:
d[100]

'影'

In [16]:
# bag of char
boc = [d.doc2bow(doc) for doc  in tqdm(x_text_sp)]

HBox(children=(IntProgress(value=0, max=71462), HTML(value='')))

In [17]:
df = pd.DataFrame([len(b) for b in boc],columns=["length"])

In [18]:
df.head()

,length
0,140
1,25
2,57
3,71
4,72


In [19]:
df.describe()

,length
count,71462.000000
mean,62.733201
std,39.017601
min,1.000000
25%,37.000000
50%,53.000000
75%,77.000000
max,730.000000


In [20]:
len(d)

3794

In [21]:
%%notify
dense = list(matutils.corpus2dense(boc,  num_terms=len(d)))

<IPython.core.display.Javascript object>

In [22]:
dense = np.array(dense).T

In [23]:
dense.shape

(71462, 3794)

In [24]:
%%notify
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense, y, test_size=0.05)

<IPython.core.display.Javascript object>

In [25]:
estimator = RandomForestClassifier(verbose=10)

In [26]:
%%notify
estimator.fit(data_train_s, label_train_s)

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.0s remaining:    0.0s


building tree 3 of 10


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   26.4s remaining:    0.0s


building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   29.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=10,
            warm_start=False)

<IPython.core.display.Javascript object>

In [27]:
estimator.score(data_test_s, label_test_s)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.8975937325125909

In [28]:
tuned_parameters = [
    {
        "n_estimators": [50, 70, 90, 110, 130, 150]
    }
]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=1,verbose=10)

In [29]:
%%notify
clf.fit(data_train_s, label_train_s)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] n_estimators=50 .................................................


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [30]:
y_true, y_pred = label_test_s, clf.predict(data_test_s)
print(accuracy_score(y_true, y_pred, target_names=["nag","pos"]))

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

# Character level + LogisticRegression

In [31]:
y_ = [v.argmax() for v in y]

In [32]:
y_[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [33]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense, y_, test_size=0.05)

In [34]:
estimator = LogisticRegression(verbose=10)

In [35]:
estimator.fit(data_train_s, label_train_s)

[LibLinear]

/Users/tdual/anaconda2/envs/py3.6/lib/python3.6/site-packages/sklearn/svm/base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=10, warm_start=False)

In [36]:
estimator.score(data_test_s, label_test_s)

0.9124230554001119

In [ ]:
diparameter={"C": [10**i for i in range(-2,4)]}
licv=GridSearchCV(LogisticRegression(),param_grid=diparameter, cv=2, scoring='accuracy', n_jobs=1,verbose=10)
licv.fit(data_train_s, label_train_s)
predictor=licv.best_estimator_

In [ ]:
y_pred[:10].tolist()

In [ ]:
y_true[:10]

In [ ]:
y_true, y_pred = label_test_s, licv.predict(data_test_s)
#print(classification_report(y_true, y_pred, target_names=["nag","pos"], digits=4))

# BOW 

In [37]:
%%notify
level = "word" 
x_text, y, ratio = load_data_and_labels(positive_data_file, negative_data_file, level=level, lang="Ja")

Japanese stopword:  あそこ, あたり, あちら ...
English stopword: ... you've, z, zero
# pos:  62402
# neg:  9060
pos/neg: 6.887637969094922


<IPython.core.display.Javascript object>

In [42]:
d = corpora.Dictionary(x_text)

In [ ]:
len(d)

In [43]:
bow =[d.doc2bow(doc) for doc  in tqdm(x_text)]

HBox(children=(IntProgress(value=0, max=71462), HTML(value='')))

In [ ]:
df = pd.DataFrame([len(b) for b in bow],columns=["length"])

In [ ]:
df.head()

In [ ]:
df.describe()

In [39]:
label =  [np.argmax(v) for v in y]

In [ ]:
len(label)

In [44]:
dense = list(matutils.corpus2dense(bow,  num_terms=len(d)))

In [ ]:
del pos_bow
del neg_bow
del pos_label
del neg_label

In [45]:
dense = np.array(dense).T

In [46]:
dense.shape

(71462, 56271)

# BOW + random forest

In [47]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense, label, test_size=0.05)

In [ ]:
estimator = RandomForestClassifier(verbose=10)

In [ ]:
estimator.fit(data_train_s, label_train_s)

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s


building tree 3 of 10


In [ ]:
estimator.score(data_test_s, label_test_s)

In [ ]:
tuned_parameters = [{'n_estimators': [50, 70, 90, 110, 130, 150]}]#, 'max_features': ['auto', 'sqrt', 'log2', None]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=1,verbose=10)

In [ ]:
clf.fit(data_train_s, label_train_s)

In [ ]:
print("best param")
print(clf.best_estimator_)

In [ ]:
for params, mean_score, all_scores in clf.grid_scores_:
        print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, all_scores.std() / 2, params))

In [ ]:
y_true, y_pred = label_test_s, clf.predict(data_test_s)
print(classification_report(y_true, y_pred,target_names=["nag","pos"]))

# BOW + LogisticRegression

In [ ]:
y_ = [np.argmax(v) for v in label]

In [ ]:
y_[:10]

In [ ]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense, y_, test_size=0.05)

In [ ]:
estimator = LogisticRegression(verbose=10)

In [ ]:
estimator.fit(data_train_s, label_train_s)

In [ ]:
estimator.score(data_test_s, label_test_s)

In [ ]:
diparameter={"C": [10**i for i in range(-2,4)]}
licv=GridSearchCV(LogisticRegression(),param_grid=diparameter, cv=2, scoring='accuracy', n_jobs=1,verbose=10)
licv.fit(data_train_s, label_train_s)
predictor=licv.best_estimator_

# Tf-Idf + random forest

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
with open("data/amazon_ja/pos.txt") as f:
    raw_doc = f.readlines()
with open("data/amazon_ja/neg.txt") as f:
    raw_doc += f.readlines()
len(raw_doc)

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=t.tokenize)
train_matrix = vectorizer.fit_transform(raw_doc)

In [ ]:
train_matrix.shape

In [ ]:
train_matrix[0]

In [ ]:
#data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense.T, label, test_size=0.1)
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(train_matrix, label, test_size=0.05)

In [ ]:
estimator = RandomForestClassifier(verbose=10)

In [ ]:
estimator.fit(data_train_s, label_train_s)

In [ ]:
estimator.score(data_test_s, label_test_s)

In [ ]:
tuned_parameters = [{'n_estimators': [50, 70, 90, 110, 130, 150]}]#, 'max_features': ['auto', 'sqrt', 'log2', None]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=3,verbose=10)

In [ ]:
clf.fit(data_train_s, label_train_s)

In [ ]:
print("best param")
print(clf.best_estimator_)

In [ ]:
for params, mean_score, all_scores in clf.grid_scores_:
        print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, all_scores.std() / 2, params))

In [ ]:
y_true, y_pred = label_test_s, clf.predict(data_test_s)
print(classification_report(y_true, y_pred,target_names=["nag","pos"]))

# Tf-Idf + LogisticRegression

In [ ]:
y_ = [np.argmax(v) for v in label]

In [ ]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(train_matrix, label, test_size=0.05)

In [ ]:
estimator = LogisticRegression(verbose=10)

In [ ]:
estimator.fit(data_train_s, label_train_s)

In [ ]:
estimator.score(data_test_s, label_test_s)

In [ ]:
diparameter={"C": [10**i for i in range(-2,4)]}
licv=GridSearchCV(LogisticRegression(),param_grid=diparameter, cv=2, scoring='accuracy', n_jobs=1,verbose=10)
licv.fit(data_train_s, label_train_s)
predictor=licv.best_estimator_